In [1]:
import torch 
from torch import nn 
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
X = torch.rand(2,20)
net(X)

tensor([[-0.1905, -0.1380,  0.0555,  0.0723,  0.1181,  0.1635,  0.3352, -0.2762,
         -0.0397, -0.1579],
        [-0.1147, -0.0477,  0.2063,  0.0972,  0.2191,  0.0822,  0.1356, -0.1861,
          0.0031, -0.1109]], grad_fn=<AddmmBackward0>)

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [7]:
net = MLP()
net(X)

tensor([[-0.0101, -0.0470,  0.1398,  0.1965,  0.0596, -0.0679, -0.0860,  0.2774,
         -0.0793,  0.2662],
        [ 0.0097,  0.0502,  0.1373,  0.0079,  0.1435, -0.0539,  0.0214,  0.1611,
         -0.1975,  0.0745]], grad_fn=<AddmmBackward0>)

In [11]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[-0.2323,  0.3271,  0.1461,  0.3713,  0.0716,  0.0178,  0.1293, -0.2017,
          0.2081,  0.1452],
        [-0.2523,  0.2383,  0.0846,  0.2369,  0.2055, -0.0072,  0.2755, -0.3014,
          0.2950,  0.0110]], grad_fn=<AddmmBackward0>)

In [17]:
class FixedHiddenMPL(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=True)
        self.linear = nn.Linear(20,20)
        
    def forward(self,X):
        X = self.linear(X)
        X = F.relu(torch.mm(X,self.rand_weight)+1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net = FixedHiddenMPL()
net(X)

tensor(-0.0577, grad_fn=<SumBackward0>)

In [18]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)
    def forward(self,X):
        return self.linear(self.net(X))
chimera = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMPL())
chimera(X)

tensor(-0.0483, grad_fn=<SumBackward0>)